In [62]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2
from entropy_bounds.utils import optimization_cf
from binaryIV import simulate_deterministic_data_with_probabilistic_ate, extract_prob_dict, entropy_of_array
from entropy_bounds.entropy_bounds import compute_ate_bounds


import matplotlib.pyplot as plt
import seaborn as sns
import math
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('display.max_columns', None)


In [11]:
data = pd.read_pickle('binaryIV_results.pkl')
data['entropy_U'].describe()

count    2000.000000
mean        0.692177
std         0.001375
min         0.680292
25%         0.691795
50%         0.692755
75%         0.693019
max         0.693147
Name: entropy_U, dtype: float64

In [ ]:
# minimal working example
sim = data.iloc[0]

df = pd.DataFrame({'Y': sim['Y'], 'X': sim['X'], 'Z': sim['Z']})

entr = 0.7

# Compute ATE bounds
ate_lower, ate_upper = compute_ate_bounds(df['X'].to_numpy(), df['Y'].to_numpy(), entr)

# Check if true ATE is within bounds
bounds_valid = ate_lower <= sim['ATE_true'] <= ate_upper

# Collect results
result = {
    'ATE_true': sim['ATE_true'],
    'bound_lower': ate_lower,
    'bound_upper': ate_upper,
    'bound_width': ate_upper - ate_lower,
    'bounds_valid': bounds_valid
}

# Display
for key, value in result.items():
    print(f"{key}: {value}")

ATE_true: -0.6494981193401556
bound_lower: -0.7589942481344406
bound_upper: 0.202100234716181
bound_width: 0.9610944828506216
bounds_valid: True


In [59]:
data['entropy_U'].describe()

count    2000.000000
mean        0.692177
std         0.001375
min         0.680292
25%         0.691795
50%         0.692755
75%         0.693019
max         0.693147
Name: entropy_U, dtype: float64

In [ ]:
df_results = data.copy()
entr = np.random.uniform(0, 1)
for idx, sim in df_results.iterrows():

    df = pd.DataFrame({'Y': sim['Y'], 'X': sim['X'], 'Z': sim['Z']})

    # Compute ATE bounds
    ate_lower, ate_upper = compute_ate_bounds(df['X'].to_numpy(), df['Y'].to_numpy(), entr, method='cf')

    # Check if true ATE is within bounds
    bounds_valid = ate_lower <= sim['ATE_true'] <= ate_upper

    # Collect results
    result = {
        'ATE_true': sim['ATE_true'],
        'bound_lower': ate_lower,
        'bound_upper': ate_upper,
        'bound_width': ate_upper - ate_lower,
        'bounds_valid': bounds_valid
    }

    df_results.at[idx, 'entropyBounds_H(conf)_UB'] = entr
    df_results.at[idx, 'entropyBounds_bound_lower'] = ate_lower
    df_results.at[idx, 'entropyBounds_bound_upper'] = ate_upper
    df_results.at[idx, 'entropyBounds_bound_width'] = ate_upper - ate_lower
    df_results.at[idx, 'entropyBounds_bounds_valid'] = bounds_valid

In [ ]:
print(f"Invalid Bounds: {df_results[df_results['entropyBounds_bounds_valid'] == False].shape[0]}")
print(f"Average Bound Width: {df_results['entropyBounds_bound_width'].mean()}")


Invalid Bounds: 4
Average Bound Width: 0.912342858446651


In [ ]:
## RCT: entropy --> bound width
random_entropy_valid = df_results[df_results['entropyBounds_bounds_valid'] == True].copy()

